In [1]:
import pandas as pd
from pathlib import Path

import sys
sys.path.insert(0, '../..')
from scripts.utils.latex import show_latex, TABLES

# Load data

In [2]:
datapath = Path('../../data')

notes = pd.read_pickle(datapath / '2020_raw/processed.pkl')
cov = pd.read_pickle(datapath / '2020_raw/ICD_U07.1/notes_[U07.1]_2020_q1_q2_q3.pkl')
annotated = pd.read_csv(path / 'annotated_notes_ids.csv', dtype={'MDN': str, 'NotitieID': str})

In [3]:
cols = ['MDN', 'CSN', 'typecontact', 'DBC-id', 'specialisme', 'episodenaam', 'DBC_diagnose', 'ICD10_diagnose']
settings = dict(sep=';', names=cols, encoding = 'utf-8')
extract_name = lambda f: str(f).split(' ')[1].lower()
dfs = {extract_name(f):pd.read_csv(f, **settings) for f in datapath.glob('Diagnoses*.csv')}
diag = pd.concat(dfs.values(), keys=dfs.keys()
).astype({'MDN':str}).reset_index(level=0
).rename(columns={'level_0': 'institution'})

# All data

In [4]:
caption = "Number of unique patients in `Diagnoses` and `Notities` files"
label = "2020_n_pat_diag_note"

pd.concat([
    notes.groupby('institution').MDN.nunique().rename('patients in `Notities`'),
    diag.groupby('institution').MDN.nunique().rename('patients in `Diagnoses`'),
    ], axis=1
).T.assign(total=lambda df: df.sum(axis=1)).rename_axis('n_unique').pipe(show_latex, caption, label)

institution,amc,vumc,total
n_unique,,,
patients in `Notities`,"56,662","43,658","100,320"
patients in `Diagnoses`,"45,154","35,654","80,808"


In [5]:
caption = "All data: num patients, num notes, num diagnoses"
label = "2020_all_overview"

pd.concat([
    notes.query("MDN.isin(@diag.MDN)").groupby('institution').MDN.nunique().rename('patients'),
    notes.query("MDN.isin(@diag.MDN)").groupby('institution').NotitieID.nunique().rename('notes'),
    diag.groupby('institution').ICD10_diagnose.nunique().rename('diagnoses'),
    ], axis=1
).T.assign(total=lambda df: df.sum(axis=1)).rename_axis('n_unique').pipe(show_latex, caption, label)

institution,amc,vumc,total
n_unique,,,
patients,"45,427","36,072","81,499"
notes,"1,287,903","987,870","2,275,773"
diagnoses,"3,272","2,783","6,055"


# COVID

### COVID-19, virus geïdentificeerd \[U07.1]

In [6]:
caption = "COVID data overview ('COVID-19, virus geïdentificeerd [U07.1]')"
label = "2020_covid_overview"

selection = annotated.query("year==2020")
piv = pd.concat([
    selection.groupby('institution')[['MDN', 'NotitieID']].nunique(),
    cov.groupby('institution')[['MDN', 'NotitieID']].nunique(),
], keys=['annotated', 'all']).rename(columns={'MDN':'patients', 'NotitieID':'notes'}
).unstack(0).stack(0).assign(rest=lambda df: df['all'] - df.annotated)

piv.append(pd.concat([piv.xs('amc') + piv.xs('vumc')], keys=['total'])).pipe(show_latex, caption, label)

all annotated    rest
institution                                   
amc         notes     22,220       941  21,279
            patients     261        31     230
vumc        notes     22,292       746  21,546
            patients     233        29     204
total       notes     44,512     1,687  42,825
            patients     494        60     434

In [7]:
prefix = '2020'
for idx, table in enumerate(TABLES):
    with open(f'./tables/{prefix}_{idx}.tex', 'w', encoding='utf8') as f:
        f.write(table)